# 꽃 이미지 분류

- 5종류의 꽃(daisy, dandelion, sunflower, rose, tulip) 이미지를 분류하는 머신러닝 모델을 만듭니다.
- K-최근접 이웃(K-Nearest Neighbors) 알고리즘을 사용합니다.

In [ ]:
import numpy as np
import os
import PIL.Image as pilimg
import imghdr
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier

## 1. 데이터 준비 (Raw Image to NPZ)

- 각 꽃 폴더에 있는 이미지 파일들을 읽어 Numpy 배열로 변환합니다.
- 이미지 크기를 (80, 80)으로 통일하고, 각 폴더에 따라 0~4까지 레이블링을 수행합니다.
- 변환된 데이터는 `daisy.npz`, `dandelion.npz` 등과 같이 꽃 종류별로 `.npz` 파일로 저장됩니다.

**참고:** 아래 `makeData`와 `filesave` 함수는 원본 이미지 파일(`.jpg`, `.png` 등)을 머신러닝에 사용하기 좋은 `.npz` 포맷으로 변환하는 전처리 과정입니다. 이 과정은 한 번만 실행하면 됩니다. 만약 `.npz` 파일들이 이미 존재한다면 이 셀은 실행할 필요가 없습니다.

In [ ]:
# makeData => 폴더명과 라벨을 주면 해당 폴더 데이터를 읽어 numpy 배열로 바꾸고 라벨링
def makeData(folder, label):
    data =[]  # 이미지의 피처를 저장
    labels=[] # 라벨 저장
    
    # './data/flowers' 경로는 실제 데이터 위치에 맞게 수정해야 할 수 있습니다.
    path = "./data/flowers/" + folder 
    
    if not os.path.isdir(path):
        print(f"경고: '{path}' 디렉토리를 찾을 수 없습니다. 경로를 확인해주세요.")
        return

    for filename in os.listdir(path):
        try:
            filepath = os.path.join(path, filename)
            kind = imghdr.what(filepath) # 파일의 종류 확인
            
            if kind in ["gif", "png", "jpg", "jpeg"]:
                img = pilimg.open(filepath)
                # 이미지 크기를 80x80으로 통일
                resize_img = img.resize((80, 80))
                pixel = np.array(resize_img)  # 이미지를 numpy 배열로 변환
                
                # 컬러 이미지만(3채널) 사용
                if pixel.shape == (80, 80, 3):
                    data.append(pixel)
                    labels.append(label)
        except Exception as e:
            print(f"파일 처리 오류: {filename}, 오류: {e}")

    print(f"'{folder}' 처리 완료. 총 {len(data)}개의 이미지.")
    # 파일로 저장하기
    np.savez("{}.npz".format(folder), data=data, targets=labels)

def filesave():
    # 1. 파일로 저장하기
    print("데이터 변환 및 저장을 시작합니다...")
    makeData("daisy", "0") 
    makeData("dandelion", "1") 
    makeData("sunflower", "2") 
    makeData("rose", "3") 
    makeData("tulip", "4")
    print("데이터 변환 및 저장 완료.")

In [ ]:
# 아래 주석을 해제하고 실행하면 이미지 파일들을 .npz 파일로 변환합니다.
# .npz 파일이 이미 있다면 실행할 필요 없습니다.
# filesave()

## 2. 데이터 로딩

- 전처리 단계에서 생성된 `.npz` 파일들을 불러옵니다.
- 각 파일을 하나로 합쳐 전체 데이터셋을 구성합니다.

In [ ]:
def loadData():
    try:
        daisy = np.load("daisy.npz")
        dandelion = np.load("dandelion.npz")
        sunflower = np.load("sunflower.npz")
        rose = np.load("rose.npz")
        tulip = np.load("tulip.npz")
    except FileNotFoundError as e:
        print(f"오류: {e}")
        print("'.npz' 파일을 찾을 수 없습니다. 이전 단계에서 'filesave()'를 실행했는지 확인해주세요.")
        return None, None

    data = np.concatenate((daisy["data"], dandelion["data"], sunflower["data"], rose["data"], tulip["data"]))
    target = np.concatenate((daisy["targets"], dandelion["targets"], sunflower["targets"], rose["targets"], tulip["targets"]))
    
    print("데이터 로딩 완료.")
    print("전체 데이터 형태:", data.shape)
    print("전체 타겟 형태:", target.shape)
    return data, target

data, target = loadData()

## 3. 데이터 전처리

- **Reshape**: Scikit-learn의 KNN 모델은 2차원 데이터를 입력으로 받습니다. 따라서 (샘플 수, 80, 80, 3) 형태의 4차원 데이터를 (샘플 수, 80 * 80 * 3) 형태의 2차원 데이터로 변환합니다.
- **Scaling**: 픽셀 값의 범위를 0~255에서 0~1 사이로 조정(정규화)합니다. 이는 모델의 학습 성능을 향상시키는 데 도움이 됩니다.

In [ ]:
if data is not None:
    # 4차원 데이터를 2차원 데이터로 변환
    # (샘플 수, 너비, 높이, 채널) -> (샘플 수, 너비*높이*채널)
    data = data.reshape(data.shape[0], 80 * 80 * 3)
    print("Reshape 이후 데이터 형태:", data.shape)

    # 데이터 스케일링 (0~1 사이로 정규화)
    data = data / 255.0
    print("데이터 스케일링 완료.")

## 4. 훈련 및 테스트 데이터 분리

- 전체 데이터셋을 모델 학습에 사용할 훈련(Train) 데이터와 모델 성능 평가에 사용할 테스트(Test) 데이터로 분리합니다.
- `train_test_split` 함수를 사용하여 50:50 비율로 나눕니다.

In [ ]:
if data is not None:
    X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.5, random_state=0)

    print("훈련 데이터셋 크기:", X_train.shape, y_train.shape)
    print("테스트 데이터셋 크기:", X_test.shape, y_test.shape)

## 5. K-최근접 이웃(KNN) 모델 학습 및 평가

- K-최근접 이웃 분류기 모델을 생성하고 훈련 데이터로 학습시킵니다.
- 학습된 모델을 사용하여 훈련 데이터와 테스트 데이터에 대한 정확도(Accuracy)를 평가합니다.

In [ ]:
if 'X_train' in locals():
    model = KNeighborsClassifier(n_neighbors=5)
    model.fit(X_train, y_train)

    train_score = model.score(X_train, y_train)
    test_score = model.score(X_test, y_test)

    print(f"훈련 세트 정확도: {train_score:.4f}")
    print(f"테스트 세트 정확도: {test_score:.4f}")